# Background
Version 1 of the PSU had a RT8279GSP (unavailable), which ended up being replaced by a pin-compatible, but expensive [TPS5410](https://www.trustedparts.com/en/part/texas-instruments/TPS5410MDREP).

There are many more and cheaper options available when another pinout for the converter is used.
The [TPS54231D](https://www.trustedparts.com/en/part/texas-instruments/TPS54231D) is much cheaper and readily available.

![Typical Application Schematic](../docs/fig10.png)

### References
* [Switching Power Supply Topology Voltage Mode vs. Current Mode](https://www.ti.com/lit/an/slua119/slua119.pdf)
* [UNDERSTANDING AND APPLYING CURRENT-MODE CONTROL THEORY](https://www.ti.com/lit/an/snva555/snva555.pdf)

In [35]:
# TPS54231 parameters
Ven = 1.25 # [V] Enable threshold voltage
Vref = 0.8 # [V] Reference aka feedback voltage
iss = 2e-6 # [A] Soft start charge current
Fsw = 570e3 # [Hz] switching frequency

# Design choices
UVLO_rising = 22 # [V] avoid loading 24V of the garage door too much
UVLO_falling = 20
css = 10e-9 # [F]
Vout = 5 # [V] output voltage setting
Iout_max = 2 # [A] max. output current
Vin_max = 28        # maximum input voltage [V]


## Under voltage calculation

In [36]:
# User preferences :
Ren1 = (UVLO_rising - UVLO_falling)/3e-6
print("Ren1 = "+str(Ren1)+"ohm")
Ren2 = Ven /((UVLO_rising - Ven)/Ren1 + 1e-6)
print("Ren2 = "+str(Ren2)+"ohm")

# UVLO calculation with e24-value resistors
Ren1_e24 = 680e3
Ren2_e24 = Ven /((UVLO_rising - Ven)/Ren1_e24 + 1e-6)
print("Ren2_e24 = "+str(Ren2)+"ohm")

Ren1 = 666666.6666666666ohm
Ren2 = 38910.505836575874ohm
Ren2_e24 = 38910.505836575874ohm


### Final results
* R1 = 680㏀
* R2 = 39㏀

In [3]:
Ven = 1.25 # [V] enable voltage
Ren1 = 680e3
Ren2 = 39e3
Vstart = (Ven/Ren2 - 1e-6)*Ren1 + Ven
print("Typical UVLO rising = " + str(Vstart))
Vstop = Vstart - Ren1 * 3e-6
print("Typical UVLO falling = "+ str(Vstop))

Typical UVLO rising = 22.364871794871796
Typical UVLO falling = 20.324871794871797


## Slow-start

In [37]:
tss = css * Vref / iss
print("Slow start time  = " + str(tss) + " s with C5 = " + str(css) + "F")

Slow start time  = 0.004 s with C5 = 1e-08F


### Output voltage set point

In [38]:
R5 = 10e3 # [Ω]
R6 = R5 * Vref / (Vout - Vref)
print(R6)

1904.7619047619046


According to [Resistor calculator (series and parallel)](https://www.qsl.net/in3otd/parallr.html) we get closest by setting : 
* R6 = 1.8㏀ + 100Ω

## Input capacitors

In [39]:
import math

Cin = 10e-6  # 10µF
Cin_count = 2 # Number of Cin caps in parallel
DF = 0.1 # 10% in Yageo datasheet

Xc = 1 /(2*math.pi*Fsw*Cin)
ESR_max = DF * Xc
delta_vin = Iout_max * 0.25 / (Cin * Cin_count * Fsw) + Iout_max * ESR_max / Cin_count
print('Input voltage ripple is {:.3}V.'.format(delta_vin))
print('RMS-ripple current is {:.3}A.'.format(Iout_max/2))

Input voltage ripple is 0.0467V.
RMS-ripple current is 1.0A.


### Final results
Ripple current shouldn't be an issue.  TDK specifies about 2°C for 1A, 570kHz RMS ripple current.
* C1 = C2 = 10µF/50V/1210
* C3 = 10nF/50V/0603

## Inductor selection

In [40]:
# Design requirements
Kind = 0.2          # ΔIL/IL
k = 0.8             # TI design margin 

# Calculated parameters
T = 1/Fsw                                     # switching period [s]
D = Vout/Vin_max                              # duty cycle
t_on = T * D                                  # MOSFET-ON period [s]
delta_IL_on = Kind * Iout_max                 # Ripple current
L_min = (Vin_max - Vout) * t_on / delta_IL_on # Minimum recommended inductor
print('Minimum inductor value is {:.2}H'.format(L_min))

L = 18e-6 # [H] Selection for output inductor

# Inductor current calculations
Ilpp = (Vin_max - Vout) * t_on / (L * k)
print('Peak-peak inductor ripple current is {:.2}A'.format(Ilpp))
Ilrms = pow(pow(Iout_max,2) + pow(Ilpp,2)/12,0.5)
print('Inductor RMS current is {:.2}A'.format(Ilrms))
Ilpk = Iout_max + Ilpp/2
print('Peak inductor current is {:.2}A'.format(Ilpk))

Minimum inductor value is 1.8e-05H
Peak-peak inductor ripple current is 0.5A
Inductor RMS current is 2.0A
Peak inductor current is 2.3A


A 12.7 x 12.7mm, shielded, <40mΩ, >3A inductor will do:
* Wuerth 744771118
* Eaton DR125-180-R
* Bourns SRR1260-180M
* Pulse P1173.183NLT
* Sumida CDRH127NP-180MC

## Output capacitor selection
Closed loop cross-over frequency must be < Fsw_min/8, but for the TPS54231 it will be limited anyhow to 25kHz.

In [41]:
ro = Vout / Iout_max
fco_max = 25e3 # [Hz] maximum desired cross-over frequency
Co_min = 1 / (2 * math.pi * ro * fco_max) # minimum output capacitance [F]
print('Minimum output capacitance is {:.2}F'.format(Co_min))

Co = 47e-6 # [F] Selected output capacitor
Co_count = 2
Co_tot = Co * Co_count
Resr_co = 5e-3 # [Ω]

# Output ripple calculation
Vopp = Ilpp * ((D-0.5)/(4*Fsw*Co_tot) + Resr_co)
print('Output voltage ripple is {:.2}V'.format(Vopp))
# Output capacitor RMS-ripple current calculation
Icout_rms = Ilpp / pow(12,0.5)
print('Output capacitor\'s current ripple is {:.2}A'.format(Icout_rms))

Minimum output capacitance is 2.5e-06F
Output voltage ripple is 0.0018V
Output capacitor's current ripple is 0.14A


### Design choice
* C8 = C9 = 47µF/10V

## Compensation network
We have the joy of using a component without an internal compensation network.
A [Type II compensation network](https://www.ti.com/lit/an/slva662/slva662.pdf) must be connected externally.  
1. Set R3 to determine the cross-over frequency
2. Set C6 to change the frequency of the zero
3. Set C7 to change the frequency of the pole

The TPS54231 datasheet uses [Roea](https://www.ti.com/lit/an/slva503a/slva503a.pdf), which is fictional load resistance of the transconductance error amplifier.
Roea = DCgain_error_amplifier / transconductance_error_amplifier = 800 / 92e-6 = 8.696MΩ

### Frequency calculation of the pole and the zero.

In [42]:
PB = 60 # [°] desired phase boost
PB_rad = PB * math.pi/180

# Calculate frequency of first pole
fp = (math.tan(PB_rad)+math.pow(math.pow(math.tan(PB_rad),2)+1,0.5))*fco_max
print('Pole frequency is {:.3}Hz'.format(fp))

# Calculate frequency of the zero (cross-over frequency is geometric mean of pole and zero)
fz = math.pow(fco_max,2) / fp
print('Zero frequency is {:.3}Hz'.format(fz))

Pole frequency is 9.33e+04Hz
Zero frequency is 6.7e+03Hz


### Compensation network components

In [43]:
Gmp = 9 # [A/V] Current sense gain, aka switch current to COMP transconductance
Gma = 92e-6 # [A/V] Error amplifier transconductance

R3 = 2 * math.pi * fco_max * Vout * Co_tot / (Vref * Gmp * Gma)
print('R3 = {:.3}Ω'.format(R3))

C6 = 1 / (2 * math.pi * fz * R3)
print('C6 = {:.3}F'.format(C6))

C7 = 1 / (2 * math.pi * fp * R3)
print('C7 = {:.3}F'.format(C7))

R3 = 1.11e+05Ω
C6 = 2.13e-10F
C7 = 1.53e-11F


## Bootstrap capacitor
100nF/0603

## Catch-diode
* Reverse voltage > Vinmax + 0.5V
* Maximum current > peak inductor current
* TI Webench advices FSV360FP